In [3]:
from utils import get_gemini_api_key
GEMINI_API_KEY = get_gemini_api_key()
#print(OPENAI_API_KEY)
llm_config = {"model": "gemini-2.5-flash","api_key":GEMINI_API_KEY,"api_type":"google"}

# Defining an autogen agent

In [4]:
from autogen import ConversableAgent

agent = ConversableAgent(
    name="chatbot",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

In [5]:
# we are testing single turn conversation using generate_replyjjjjj
reply = agent.generate_reply(
    messages=[{"content": "Tell me a joke.", "role":"user"}]
)
print(reply)

{'content': 'Why did the scarecrow win an award?\n\nBecause he was outstanding in his field!', 'refusal': None, 'role': 'assistant', 'annotations': None, 'audio': None, 'function_call': None, 'tool_calls': None}


# Conversation

# Setting up a conversation between two agents, Cathy and Joe, where the memory of their interactions is retained.

In [6]:
cathy= ConversableAgent( name= "cathy",
                         system_message="Your name is Cathy and you are a stand-up comedian.",
                         llm_config=llm_config,
                         human_input_mode="NEVER"
                       )

joe = ConversableAgent( name="joe",
                       system_message="Your name is Joe and you are a stand-up comedian. "
                       "Start the next joke from the punchline of the previous joke.",
                       llm_config=llm_config,
                       human_input_mode="NEVER",
                      )

In [7]:
chat_result = joe.initiate_chat(
    recipient=cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.",
    max_turns=2,
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Alright Joe, you got it! The jokes are *never* stopping when I'm on stage... or, you know, just talking to people.

So, I was trying to adult the other day, right? Like, really *adult*. I decided I was going to conquer my laundry. I opened the washing machine, looked at the pile, and then I realized... I don't actually know which one is the "casual" cycle. Is it for casual clothes? Or casual washing? Like, a "whatever" setting? I ended up just hitting "heavy duty" on everything, including my delicates. Figured if it can survive my life, it can survive a heavy duty wash.

I’m telling you, I live on the edge! Thank you, thank you, I'll be here all week! Try the veal!

--------------------------------------------------------------------------------
joe (to cathy):

"Figured if it can survive my life, it can survive a heavy duty w

In [8]:
import pprint

pprint.pprint(chat_result.chat_history)

[{'content': "I'm Joe. Cathy, let's keep the jokes rolling.",
  'name': 'joe',
  'role': 'assistant'},
 {'content': "Alright Joe, you got it! The jokes are *never* stopping when I'm "
             'on stage... or, you know, just talking to people.\n'
             '\n'
             'So, I was trying to adult the other day, right? Like, really '
             '*adult*. I decided I was going to conquer my laundry. I opened '
             'the washing machine, looked at the pile, and then I realized... '
             'I don\'t actually know which one is the "casual" cycle. Is it '
             'for casual clothes? Or casual washing? Like, a "whatever" '
             'setting? I ended up just hitting "heavy duty" on everything, '
             'including my delicates. Figured if it can survive my life, it '
             'can survive a heavy duty wash.\n'
             '\n'
             "I’m telling you, I live on the edge! Thank you, thank you, I'll "
             'be here all week! Try the ve

In [9]:
pprint.pprint(chat_result.cost)

{'usage_excluding_cached_inference': {'gemini-2.5-flash': {'completion_tokens': 408,
                                                           'cost': 0.0011832,
                                                           'prompt_tokens': 544,
                                                           'total_tokens': 952},
                                      'total_cost': 0.0011832},
 'usage_including_cached_inference': {'gemini-2.5-flash': {'completion_tokens': 408,
                                                           'cost': 0.0011832,
                                                           'prompt_tokens': 544,
                                                           'total_tokens': 952},
                                      'total_cost': 0.0011832}}


In [10]:
pprint.pprint(chat_result.summary)

('Joe! "Heavy duty panic cycle" – oh my god, you just described my default '
 'setting! My financial advisor is just a series of increasingly concerned '
 'emojis in my texts at this point. He just sends me a shrugging guy and then '
 'a sad face whenever I call.\n'
 '\n'
 'I told him, "Look, if my *brain* can survive another Tuesday morning '
 "meeting, it can survive whatever the market throws at us! We're basically "
 'just free-falling with style!" My current investment strategy is really just '
 '"buy more coffee." It feels like a sound investment in my own survival, at '
 'least. And a *heavy duty* one, for sure!\n'
 '\n'
 "You're a natural, Joe! Keep 'em coming!")


# Get a better summary of the conversation

In [11]:
chat_result = joe.initiate_chat(
    cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.", 
    max_turns=2, 
    # View autogen documentation for conversable agent
    summary_method="reflection_with_llm", #This is gives a better summary of the coversation history
    summary_prompt="Summarize the conversation",
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Alright, Joe! You got it! *[Adjusts imaginary mic, big smile]* Always ready to keep 'em rolling!

You know, I was just thinking about how quickly time flies these days. It’s like… one minute you’re worried about finding *the* perfect outfit for a Friday night, right? All the planning, the accessorizing…

The next minute, you’re just worried about finding your *reading glasses* to see the TV *on* a Friday night. And that perfect outfit? Yeah, it’s probably sweatpants now. And you know what? No regrets! Comfort is a superpower, people. Especially on a Friday. *[winks]*

--------------------------------------------------------------------------------
joe (to cathy):

Cathy, you're right, comfort *is* a superpower! My superpower used to be drinking three Red Bulls and staying up 'til dawn. Now? Now the only thing shrinking in my l

In [12]:
pprint.pprint(chat_result.summary)

('Joe and Cathy humorously commiserate about the realities of aging, where '
 'their "superpowers" have shifted from late-night activities (like Red Bulls '
 "and staying up 'til dawn) to valuing comfort and early bedtimes. They lament "
 'the "9 PM problem," where all the interesting, joke-worthy stories happen '
 "after they're already asleep—often with Joe meticulously arranging his CPAP "
 'mask. Both appeal to the audience for wild, post-9 PM anecdotes to inspire '
 'their comedy material, highlighting the challenge of finding fresh content '
 "when they're in bed so early.")


# Chat Termination

# Chat can be terminated using a termination conditions.

In [30]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian. ",
    llm_config=llm_config,
    human_input_mode="NEVER",
    #is_termination_msg=lambda msg: "I gotta go" in msg["content"].lower() or "Good bye" in msg["content"].lower(),
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"].lower() or "Good bye" in msg["content"].lower(),
)

#lamda function is broken here so the agent doesn't terminate even after the stop word is mentioned in the message conversation

In [31]:
chat_result = joe.initiate_chat(
    recipient=cathy,
    message=" Cathy, tell me a joke and then say the words 'I gotta go'..",
    #max_turns=2, 
    #summary_method="reflection_with_llm",
    #summary_prompt="Summarize the conversation"
)

joe (to cathy):

 Cathy, tell me a joke and then say the words 'I gotta go'..

--------------------------------------------------------------------------------
cathy (to joe):

Sure! Why did the scarecrow win an award? Because he was outstanding in his field! 

I gotta go!

--------------------------------------------------------------------------------
joe (to cathy):

Classic! I love that one! You know, scarecrows are the overachievers of the farm world. Picture them standing around all day, just waiting for someone to give them a pat on the back. “Hey, look at me! I’m really nailing this whole ‘looking scary’ thing!” 

But hey, when you gotta go, you gotta go! Got to keep the field clear for more laughter! 

--------------------------------------------------------------------------------
cathy (to joe):

Haha, absolutely! Scarecrows really do take their jobs seriously—it's like they studied at Scarecrow University! You can just picture them at graduation, tossing their hats in the a

KeyboardInterrupt: 

In [37]:
def debug_termination(msg):
    content = msg.get("content", "")
    # This print will show up in your console for every message
    print(f"--- DEBUG: Checking message: '{content}' ---")
    
    found = "i gotta go" in content.lower()
    print(f"--- DEBUG: Termination found? {found} ---")
    
    return found

cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=debug_termination,
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian."
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=debug_termination,
)

In [38]:
chat_result = joe.initiate_chat(
    recipient=cathy,
    message=" Cathy, tell me a joke and then end the conversation immediately.",
    #max_turns=2, 
    #summary_method="reflection_with_llm",
    #summary_prompt="Summarize the conversation"
)

joe (to cathy):

 Cathy, tell me a joke and then end the conversation immediately.

--------------------------------------------------------------------------------
cathy (to joe):

Sure, here's a joke for you: Why did the scarecrow win an award? Because he was outstanding in his field! 

Alright, I gotta go!

--------------------------------------------------------------------------------
joe (to cathy):

That's a classic! You know, scarecrows really know how to stand out, don't they? I gotta go!

--------------------------------------------------------------------------------
cathy (to joe):

Haha, right? They really do know how to make a scene! Well, I gotta go!

--------------------------------------------------------------------------------
joe (to cathy):

I love it! A scarecrow with a flair for the dramatic! Alright, I gotta go!

--------------------------------------------------------------------------------


KeyboardInterrupt: 

In [36]:
chat_result.chat_history

[{'content': ' Cathy, tell me a joke and then end the conversation immediately.',
  'role': 'assistant',
  'name': 'joe'},
 {'content': 'Sure thing! Here’s a joke for you:\n\nWhy don’t scientists trust atoms? \n\nBecause they make up everything!\n\nI gotta go!',
  'role': 'user',
  'name': 'cathy'}]

In [13]:
# MODIFYING LAMBDA FUNCTION INORDER TO TERMINATE THE CONVERSATION BASED ON STOP WORDS

cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg= lambda x: x.get("content") is not None and "i gotta go" in x["content"].lower(),
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian."
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg= lambda x: x.get("content") is not None and "i gotta go" in x["content"].lower(),
)

THE MESSAGE CONVERSATION WILL BE IN KIND OF DICTIONARY FORMAT

so message.get("content") will fetch the message received and the following process will check if its none or contains a stopword that is provided
if yes then conversation is terminated

In [14]:
chat_result = joe.initiate_chat(
    recipient=cathy,
    message=" Cathy, tell me a joke and then end the conversation immediately.",
    #max_turns=2, 
    #summary_method="reflection_with_llm",
    #summary_prompt="Summarize the conversation"
)

joe (to cathy):

 Cathy, tell me a joke and then end the conversation immediately.

--------------------------------------------------------------------------------
cathy (to joe):

Why did the scarecrow win an award? Because he was outstanding in his field!

I gotta go.

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (44ffe591-a8cc-4a96-b38b-0e7f5b6bf8b6): Termination message condition on agent 'joe' met


In [15]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "i gotta go" in str(msg.get("content") or "").lower(),
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian."
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "i gotta go" in str(msg.get("content") or "").lower(),
)

In [16]:
chat_result = joe.initiate_chat(
    recipient=cathy,
    message=" Cathy, tell me a joke and then end the conversation immediately.",
    #max_turns=2, 
    #summary_method="reflection_with_llm",
    #summary_prompt="Summarize the conversation"
)

joe (to cathy):

 Cathy, tell me a joke and then end the conversation immediately.

--------------------------------------------------------------------------------
cathy (to joe):

Why don't scientists trust atoms?

Because they make up everything!

Alright, I gotta go!

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (60b84299-25ec-4ba2-bdc2-0dfc2d0c10ad): Termination message condition on agent 'joe' met


In [17]:
chat_result.cost

{'usage_including_cached_inference': {'total_cost': 6.82e-05,
  'gemini-2.5-flash': {'cost': 6.82e-05,
   'prompt_tokens': 44,
   'completion_tokens': 22,
   'total_tokens': 66}},
 'usage_excluding_cached_inference': {'total_cost': 6.82e-05,
  'gemini-2.5-flash': {'cost': 6.82e-05,
   'prompt_tokens': 44,
   'completion_tokens': 22,
   'total_tokens': 66}}}

The "Safety Net" Example
Let's look at three different scenarios of what msg (the message object) might look like and how the lambda handles them.

# Scenario A: The phrase is present
Python
msg = {"content": "It was nice talking, but i gotta go!"}
msg.get("content") → Returns "It was nice talking, but i gotta go!".

or "" → It stays the same (it’s not empty).

.lower() → "it was nice talking, but i gotta go!".

"i gotta go" in ... → True. (The conversation STOPS).

# Scenario B: The phrase is missing
Python
msg = {"content": "Hello there!"}
msg.get("content") → Returns "Hello there!".

.lower() → "hello there!".

"i gotta go" in ... → False. (The conversation CONTINUES).

# Scenario C: The "Empty" Danger Zone (Why the logic is clever)
Imagine an agent sends a system update that has no content at all:

Python
msg = {"role": "system"} # No "content" key!
msg.get("content") → Returns None.

None or "" → This is the magic part. Python sees None is "falsy" and chooses the empty string "" instead.

str("").lower() → Results in "".

"i gotta go" in "" → False. (The code DOES NOT crash, and the conversation continues).

In [9]:
dir(chat_result)

['__annotations__',
 '__class__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__match_args__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'chat_history',
 'chat_id',
 'cost',
 'human_input',
 'summary']